<a href="https://colab.research.google.com/github/SeanBarnier/HAFS_Air-Sea/blob/main/argoComparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Retrieves and subsets Argo float data using argopy and plots it alongslide MOM6 output along the track of a TC.  

After attempting to install argopy, you may need to restart your runtime and run again.

#Set up environment

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install argopy
!pip install cartopy

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime as dt
from argopy import DataFetcher
import cartopy.crs as ccrs

#User parameters

In [ ]:
name = "Milton"
tcNum = "14"
trackType = ""

runTime = dt(year=2024, month=10, day=7, hour=6) #1 day before Milton began its most rapid intensification
singleProf = True

fHourStep = 6  #Normally 3 for HAFS-A
forecastLength = 60 #Normally 126 for HAFS-A.

figureSuffix = "_RI"
subfolder = "RI/"
dataPath = "/content/drive/MyDrive/savedData/"
figurePath = "/content/drive/MyDrive/figures/"

potentialTemp = True #Use atmospheric potential temperature instead of in-situ temperature

oceDepth = 100 # in m
maxArgoRange = 5 # in degrees

offset = "center" #Must be center, NE, SE, SW, or NE
offsetDist = 0.25

#Retrieve HAFS-A Data

Find times needed

In [ ]:
dateFormat = "%Y-%m-%d %H:%M:%S"
runFormat = "%Y%m%d_%H"
initStr, initHour = runTime.strftime(runFormat).split("_")

fcastTimes = []
for fHour in range(0, forecastLength+1, fHourStep):
  fcastTimes.append(runTime + pd.Timedelta(hours=fHour))

OR Set times manually

In [ ]:
fcastTimes = [dt(year=2024, month=10, day=7, hour=6), dt(year=2024, month=10, day=9, hour=0)]

In [ ]:
if singleProf: fcastTimes = [runTime]

Get TC location from ATCF files

In [ ]:
cols = ["BASIN", "CY", "YYYYMMDDHH", "TECHNUM/MIN", "TECH", "TAU", "LatN/S", "LonE/W",
    "VMAX", "MSLP", "TY", "RAD", "WINDCODE", "RAD1", "RAD2", "RAD3", "RAD4",
    "POUTER", "ROUTER", "RMW", "GUSTS", "EYE", "SUBREGION", "MAXSEAS", "INITIALS",
    "DIR", "SPEED", "STORMNAME", "DEPTH", "SEAS", "SEASCODE", "SEAS1", "SEAS2",
    "SEAS3", "SEAS4", "USERDEFINED1", "Thermo1", "Thermo2", "Thermo3", "Thermo4",
    "Thermo5", "Thermo6", "Thermo7", "USERDEFINED2", "DT", "SHR82", "SHR81_1",
    "SHR82_2",  "USERDEFINED3", "SST", "USERDEFINED4", "ARMW1", "ARMW2"]

In [ ]:
tcLocs = {}
windRadKey = {"NE":"RAD1", "SE":"RAD2", "SW":"RAD3", "NW":"RAD4"}
windRads = {}

atcfURL = f"https://noaa-nws-hafs-pds.s3.amazonaws.com/hfsa/{initStr}/{initHour}/{tcNum}l.{initStr}{initHour}.hfsa.trak.atcfunix"
atcfFile = "atcf_" + initStr + "_" + initHour + ".csv"
!wget -O {atcfFile} {atcfURL}
atcf = pd.read_csv(atcfFile, names=cols)

for valid in fcastTimes:

  fHour = int((valid-runTime).total_seconds() / 3600)
  pointLat = int(atcf[atcf.TAU==fHour]["LatN/S"].iloc[0].replace("N", ""))/10
  pointLon = int(atcf[atcf.TAU==fHour]["LonE/W"].iloc[0].replace("W", ""))/-10 #Assume western hemisphere
  tcLocs[valid] = (pointLat, pointLon)

Determine desired argo floats

In [ ]:
argoLoader = DataFetcher()
argoData = argoLoader.region([-95,-80,20,25,0,oceDepth,'2024-10','2024-11']).to_dataframe()

In [ ]:
argos = {}
for valid, tcLoc in tcLocs.items():
  lat = tcLoc[0] + offsetDist * (1 if offset in ["NE", "SE"] else -1)
  lon = tcLoc[1] + offsetDist * (1 if offset in ["NE", "NW"] else -1)
  argoDists = ((argoData.LATITUDE-lat)**2 + (argoData.LONGITUDE-lon)**2)**0.5
  closeArgos = argoData[argoDists < maxArgoRange]
  argoTimeDiffs = np.abs(closeArgos.TIME-valid)
  chosenArgo = closeArgos[argoTimeDiffs==np.min(argoTimeDiffs)]
  if np.min(argoTimeDiffs) > pd.Timedelta(hours=12): print(f"WARNING: Time difference at {valid} over 12 hours.")
  argos[valid] = chosenArgo

Get data from HAFS-A output.

In [ ]:
oce = {}

for valid, tcLoc in tcLocs.items():
  oce[valid] = {}

  initTime = runTime.strftime("%Y%m%d%H")
  fhour = str(int((valid-runTime).total_seconds() / 3600))
  while len(fhour) < 3: fhour = "0" + fhour
  oceFile = "mom6_" + initTime + "_f" + fhour + ".nc"
  ocePath = dataPath + "mom6Output/" + subfolder + oceFile

  if oceFile == 'mom6_2024100800_f000.nc': continue #This file is missing

  oceData = xr.open_dataset(ocePath, decode_times=False)

  lat = tcLoc[0] + offsetDist * {"NE":1, "NW":1, "SE":-1, "SW":-1, "center":0}[offset]
  lon = tcLoc[1] + offsetDist * {"NE":1, "NW":-1, "SE":1, "SW":-1, "center":0}[offset]
  oce[valid] = oceData.sel(yh=np.unique(argos[valid].LATITUDE)[0], xh=np.unique(argos[valid].LONGITUDE)[0], method="nearest").temp

#Figures

Plot modeled and in-situ ocean temperature

In [ ]:
profFig, profAxes = plt.subplots(1, len(fcastTimes), figsize=(len(fcastTimes)*2, 5))
if len(fcastTimes) == 1: profAxes = [profAxes]

for valid, profAx in zip(fcastTimes, profAxes):
  modelProf = profAx.plot(oce[valid].data[0], oce[valid].z_l.data, color="red")
  argoProf = profAx.plot(argos[valid].TEMP, argos[valid].PRES, color="blue")

  profAx.set_ylim(80, 0)
  profAx.set_xlim(25, 31)
  profAx.grid(alpha=0.5)
  profAx.set_title(valid.strftime("%m-%d %HUTC"))

modelProf[0].set_label("Model")
argoProf[0].set_label("In-Situ")
profFig.legend(loc=[0.1,-0], ncol=2)
profFig.supylabel("Depth (m)/Pressure (dBar)")
#profFig.supxlabel("Temperature ($\degree$C)")
profFig.suptitle(offset)
profFig.tight_layout()

Plot location and timing of Argo float observations

In [ ]:
mapFig = plt.figure(figsize=(6, 4))
mapAx = mapFig.add_axes([0.1,0.1,0.8,0.8], projection=ccrs.PlateCarree())
#timeAx = mapFig.add_axes([0.1,0.75,0.8,0.1])

mapAx.scatter([tcLocs[valid][1] for valid in fcastTimes], [tcLocs[valid][0] for valid in fcastTimes], color="red", s=50, transform=ccrs.PlateCarree(), label="TC Location")
mapAx.scatter([np.unique(argos[valid].LONGITUDE) for valid in fcastTimes], [np.unique(argos[valid].LATITUDE) for valid in fcastTimes], color="blue", s=50, transform=ccrs.PlateCarree(), label="Argo Float")
mapAx.coastlines()
mapAx.gridlines(draw_labels=["left", "bottom"])
mapAx.legend()

#timeAx.scatter(fcastTimes, [1]*len(fcastTimes), color="red")
#timeAx.scatter([argos[valid].TIME.iloc[0] for valid in fcastTimes], [1]*len(fcastTimes), color="blue", s=150)